<a href="https://colab.research.google.com/github/erco99/PADI-progetto/blob/main/progetto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predizione temperatura media della superficie di un Paese in un determinato periodo

**Corso di laurea triennale di Ingegneria e Scienze Informatiche, Università di Bologna, sede di Cesena**


*Programmazione di Applicazioni Data Intensive*


*Progetto d'esame di Francesco Ercolani*

# 1a. Descrizione del problema e comprensione dei dati

L'obiettivo di questo progetto è di realizzare un modello che, dato un Paese ed un periodo di tempo, sia in grado di predire la temperatura media della superficie in gradi Celsius.


Per fare ciò, si utilizzano i dati raccolti dal 1750 ad oggi.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os.path

# 1b. Analisi esplorativa dei dati

In [ ]:
file = "./GlobalLandTemperaturesByCountry.csv"
if not os.path.exists(file):
  print("problema")

temp = pd.read_csv(file, index_col=0)